In [22]:
import os
import pandas as pd

In [27]:
res_pth = r"C:\Users\pechen\Documents\XTMF\Projects\407 Model Update\407 Model - Warm Start - MCSpatial Run 35 - 20210608\Microsim Results"
zone_pth = r"C:\Users\pechen\Documents\407 Model Update"

zone_map = pd.read_csv(os.path.join(zone_pth,"407_ZoneMapping_spicy.csv"))
tripsmode_df = pd.read_csv(os.path.join(res_pth,"trip_modes.csv"))
trips_df = pd.read_csv(os.path.join(res_pth,"trips.csv"))
persons_df = pd.read_csv(os.path.join(res_pth,"persons.csv"))

In [28]:
trips_df = trips_df.merge(persons_df, on = ['household_id', 'person_id'])
trips_df = trips_df[trips_df.age >= 11]
trips_df = trips_df.merge(zone_map, left_on = 'o_zone', right_on = 'GTA06').drop(columns = "GTA06").rename(columns = {'phf_group':'subregion_o'})
trips_df = trips_df.merge(zone_map, left_on = 'd_zone', right_on = 'GTA06').drop(columns = "GTA06").rename(columns = {'phf_group':'subregion_d'})

In [29]:
trips_time_df = tripsmode_df.merge(trips_df, how = 'right', on = ['household_id', 'person_id', 'trip_id'])
trips_time_df["weight"] = trips_time_df["weight"]/10
trips_time_df["weight_z"] = trips_time_df["weight"]*trips_time_df["weight_x"]

In [30]:
am_trips_df = trips_time_df[(trips_time_df.o_depart >= 360) & (trips_time_df.o_depart < 540)]
pm_trips_df = trips_time_df[(trips_time_df.o_depart >= 900) & (trips_time_df.o_depart < 1140)]

In [31]:
am_trips = pd.pivot_table(am_trips_df, index = ['subregion_o', 'subregion_d'], columns = 'mode', values = 'weight_z', aggfunc = sum, margins = True).fillna(0)
pm_trips = pd.pivot_table(pm_trips_df, index = ['subregion_o', 'subregion_d'], columns = 'mode', values = 'weight_z', aggfunc = sum, margins = True).fillna(0)

In [32]:
am_trips['drive'] = am_trips['Auto'] + am_trips['Passenger'] + am_trips['Carpool'] + am_trips['RideShare']
pm_trips['drive'] = pm_trips['Auto'] + pm_trips['Passenger'] + pm_trips['Carpool'] + pm_trips['RideShare']

In [33]:
am_out = am_trips.filter(['Auto'])
pm_out = pm_trips.filter(['Auto'])
pm_out.to_csv(os.path.join(zone_pth, 'pm_subreg_out_run35.csv'))
am_out.to_csv(os.path.join(zone_pth, 'am_subreg_out_run35.csv'))